In [1]:
import pandas as pd
# anno_1 = '../Human_anno/humananno_res/lsy_scene.xlsx'
# anno_2 = '../Human_anno/humananno_res/lsy_scene.xlsx'

# anno_1oc =pd.read_excel(anno_1, usecols=['Model','scene'])
# anno_2oc =pd.read_excel(anno_2, usecols=['Model','scene'])
# anno_1oc = anno_1oc.astype(object).where(pd.notnull(anno_1oc), None)
# anno_2oc = anno_2oc.astype(object).where(pd.notnull(anno_2oc), None)

In [2]:
import numpy as np

def rank_numbers(numbers):
    sorted_indices = sorted(range(len(numbers)), key=lambda k: numbers[k], reverse=True)
    ranks = [0] * len(numbers)

    i = 0
    while i < len(sorted_indices):
        value_indices = [i]
        while i + 1 < len(sorted_indices) and numbers[sorted_indices[i]] == numbers[sorted_indices[i + 1]]:
            i += 1
            value_indices.append(i)
        average_rank = np.mean([index + 1 for index in value_indices])
        for index in value_indices:
            ranks[sorted_indices[index]] = average_rank
        i += 1

    return ranks

def calculate_spearman_manual(values1, values2):
    n = len(values1)
    m = len(values2)
    rank1 = rank_numbers(values1)
    rank2 = rank_numbers(values2)
    d = np.array(rank1) - np.array(rank2)
    d_squared = np.square(d)
    spearman_corr = 1 - (6 * np.sum(d_squared)) / (n * (n**2 - 1))
    return spearman_corr

def rank_basescore(base_score, draw_ratio):
    # 计算最大值和最小值的差值
    score_range = max(base_score) - min(base_score)
    # 计算平序的阈值
    draw_gap = draw_ratio * score_range

    # 对列表进行排序并保留原始索引
    indexed_scores = list(enumerate(base_score))
    indexed_scores.sort(key=lambda x: x[1])

    # 处理平序
    ranks = [0] * len(base_score)
    current_rank = 1
    for i in range(len(indexed_scores)):
        if i > 0 and abs(indexed_scores[i][1] - indexed_scores[i - 1][1]) < draw_gap:
            ranks[indexed_scores[i][0]] = current_rank
        else:
            current_rank = i + 1
            ranks[indexed_scores[i][0]] = current_rank

    return ranks

In [3]:
dimension = 'color'

In [4]:
import json
jsonpath = '../Human_anno/{}.json'.format(dimension)
with open(jsonpath,'r') as f:
    oc = json.load(f)

# history ="../GPT4o_eval_results/{}_gpt4eval_results.json".format(dimension)
# with open(history,'r') as f:
#     gpt4o_eval_history = json.load(f)

In [5]:
# gptvsanno1_spearman = 0
# gptvsanno2_spearman = 0
# anno1vsanno2_spearman = 0
# gptvsannomean = 0
# badeval = []

# for i in range(length):
#     if i % 3 == 0:
#         gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 = human_anno[:,0]
#         anno2 = human_anno[:,1]
#         annomean = (anno1 + anno2)/2
#     else:
#         gpt4o_eval_rs+= np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 += human_anno[:,0]
#         anno2 += human_anno[:,1]
#         annomean += (anno1 + anno2)/2

#     if i % 3 == 2:
#         gptvsanno1_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno1)
#         gptvsanno2_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno2)
#         anno1vsanno2_spearman += calculate_spearman_manual(anno1,anno2)
#         gptvsannomean += calculate_spearman_manual(gpt4o_eval_rs,annomean)

# gptvsanno1_spearman = 3*gptvsanno1_spearman/length
# gptvsanno2_spearman = 3*gptvsanno2_spearman/length
# anno1vsanno2_spearman = 3*  anno1vsanno2_spearman/length
# gptvsannomean = 3*gptvsannomean/length
# print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman)
# print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman)
# print("Anno1 vs Anno2 Spearman: ",anno1vsanno2_spearman)
# print("GPT vs AnnoMean Spearman: ",gptvsannomean)

In [6]:
models = ['cogvideox5b','gen3', 'kling','videocrafter2', 'pika', 'show1', 'lavie']
# models = ['videocrafter2', 'pika', 'show1', 'lavie']
# models = ['cogvideox5b','gen3', 'kling']
idexls = []
for i in range(0,len(oc)):
    idexls.append(i)
# for i in range(1,len(oc),3):
#     idexls.append(i)
length = len(idexls)

In [7]:
gptvsanno1_spearman = np.zeros(length)
gptvsanno2_spearman = np.zeros(length)
anno1vsanno2_spearman = np.zeros(length)
gptvsannomean_spearman = np.zeros(length)
baselinevsanno1_spearman = np.zeros(length)
baselinevsanno2_spearman = np.zeros(length)
baselinevsannomean_spearman = np.zeros(length)

gptscore = np.zeros([len(models)])
anno1score = np.zeros([len(models)])
anno2score = np.zeros([len(models)])
annomeanscore =np.zeros([len(models)])
baseline_rank = np.zeros([7])
badeval = []

for j in range(length):
    i = idexls[j]
    gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
    human_anno = np.array(list(oc[i]['human_anno'].values()))
    baseline_score = np.array(list(oc[i]['baseline_score'].values()))


    anno1 = human_anno[:,0]
    anno2 = human_anno[:,2]

    annomean = (anno1 + anno2)/2


    gptscore += gpt4o_eval_rs
    anno1score += anno1
    anno2score += anno2
    annomeanscore += annomean
    baseline_rank += rank_basescore(baseline_score, 0.2)


    gptvsanno1 = calculate_spearman_manual(gpt4o_eval_rs,anno1)
    gptvsanno2 = calculate_spearman_manual(gpt4o_eval_rs,anno2)
    anno1vsanno2 = calculate_spearman_manual(anno1,anno2)
    gptvsannomean = calculate_spearman_manual(gpt4o_eval_rs,annomean)
    baselinevsanno1 = calculate_spearman_manual(baseline_rank,anno1)
    baselinevsanno2 = calculate_spearman_manual(baseline_rank,anno2)
    baselinevsannomean = calculate_spearman_manual(baseline_rank,annomean)

    # j = i
    gptvsanno1_spearman[j] = gptvsanno1
    gptvsanno2_spearman[j] = gptvsanno2
    anno1vsanno2_spearman[j] = anno1vsanno2
    gptvsannomean_spearman[j] = gptvsannomean
    baselinevsanno1_spearman[j] = baselinevsanno1
    baselinevsanno2_spearman[j] = baselinevsanno2
    baselinevsannomean_spearman[j] = baselinevsannomean

    if gptvsanno1 <0.2:
        badeval.append(i)

gptscore = gptscore/length
anno1score = anno1score/length
anno2score = anno2score/length
annomeanscore = annomeanscore/length

print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman.mean())
print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman.mean())
print("Anno1 vs Anno2 Spearman: ",anno1vsanno2_spearman.mean())
print("GPT vs AnnoMean Spearman: ",gptvsannomean_spearman.mean())
print("Baseline vs Anno1 Spearman: ",baselinevsanno1_spearman.mean())
print("Baseline vs Anno2 Spearman: ",baselinevsanno2_spearman.mean())
print("Baseline vs AnnoMean Spearman: ",baselinevsannomean_spearman.mean())


print("GPT average score: ",gptscore)
print("Anno1 average score: ",anno1score)
print("Anno2 average score: ",anno2score)
print("AnnoMean average score: ",annomeanscore)

GPT vs Anno1 Spearman:  0.7971638655462185
GPT vs Anno2 Spearman:  0.8200280112044818
Anno1 vs Anno2 Spearman:  0.819187675070028
GPT vs AnnoMean Spearman:  0.7941526610644257
Baseline vs Anno1 Spearman:  0.254656862745098
Baseline vs Anno2 Spearman:  0.3202731092436975
Baseline vs AnnoMean Spearman:  0.20108543417366945
GPT average score:  [2.83921569 2.83529412 2.87843137 2.88235294 2.37647059 2.80784314
 2.74509804]
Anno1 average score:  [2.94901961 2.87843137 2.83921569 2.89803922 2.35294118 2.75686275
 2.75294118]
Anno2 average score:  [2.95686275 2.9372549  2.89803922 2.91372549 2.30980392 2.87058824
 2.79215686]
AnnoMean average score:  [2.95294118 2.90784314 2.86862745 2.90588235 2.33137255 2.81372549
 2.77254902]


In [8]:
from collections import Counter
badeval_prompt = []
for i in badeval:
    # print("spearman",gptvsannomean_spearman[i])
# print("videos",oc[i]['videos']['gen2'])
    # print("prompt",oc[i]['prompt_en'])
    badeval_prompt.append(oc[i]['prompt_en'])
    print(i%3,i,oc[i]['prompt_en'],gptvsanno1_spearman[i])
    # print("GPT score",oc[i]['gpt4o_eval'])
    # print("Anno score",oc[i]['human_anno'])
    # print("GPT reasons",gpt4o_eval_history[str(i)])

0 213 An orange clock 0.125
0 222 A black clock -0.0625


In [9]:
conuter = Counter(badeval_prompt)
repeated = {k:v for k,v in conuter.items() if v>1}

In [10]:
# print(sum(repeated.values()))
# print(repeated)
# with open('D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\BadEval4OverallConsistency.json','w') as f:
#     json.dump(repeated,f,indent=4)